# Detecting roads, buildings, airpots and railways using OSDataHub API

In [1]:
!pip install fiona
!pip install folium
!pip install geopandas



[notice] A new release of pip available: 22.1.2 -> 22.2.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip available: 22.1.2 -> 22.2.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip available: 22.1.2 -> 22.2.1
[notice] To update, run: pip install --upgrade pip


In [2]:
import geopandas as gpd
import requests
import json


In [3]:
def get_feature_type_in_bbox(bbox, feature_type):
        
    wfs_endpoint = ('https://api.os.uk/features/v1/wfs?')

    service = 'wfs'
    request = 'GetFeature'
    version = '2.0.0'
    typeNames = feature_type
    outputFormat = 'GEOJSON'

    params_wfs = {'service':service,
                  'key': 'SYftl2NvP35BS3NTa6MIuwv5Jo0meebK',
                  'request':request,
                  'version':version,
                  'typeNames':typeNames,
                  'outputFormat':outputFormat,
                  'bbox': bbox,
                 }

    try:
        r = requests.get(wfs_endpoint, params=params_wfs)
        r.raise_for_status()
    except requests.exceptions.RequestException as e:
        print(e)
   
    payload = r.json()
    
    return payload



## Bbox examples

In [4]:
heathrow_bbox = '51.425489,-0.578837,51.509863,-0.331644'
luton_bbox = "51.864354,-0.411768,51.890027,-0.329370"
lake_bbox = "55.178886,-2.511239,55.190757,-2.470040"
field_bbox = "52.128174,-1.159797,52.131365,-1.149498"
forest_bbox = "56.989911,-3.846481,56.992743,-3.836181"

## Airports

In [5]:
def get_airports_in_bbox(bbox):
    payload = get_feature_type_in_bbox(bbox, 'Zoomstack_Airports')
    # WGS 84
    WGS84 = 'epsg:4326'
    # British National Grid
    BNG = 'epsg:27700'

    gdf_boundary = gpd.GeoDataFrame.from_features(payload['features'], crs=WGS84)
    return gdf_boundary

geo_df = get_airports_in_bbox(heathrow_bbox)
print(geo_df)

                            geometry  OBJECTID                     Name
0  POINT (507325.86000 175741.92000)        43  London Heathrow Airport


In [6]:
def is_airport(bbox):
    result = get_feature_type_in_bbox(bbox, 'Zoomstack_Airports')
    if len(result['features']) > 0:
        return True
    else:
        return False
    
print(is_airport(heathrow_bbox))
print(is_airport(luton_bbox))
print(is_airport(lake_bbox))

True
True
False


## Water

In [7]:
def is_water(bbox):
    result = get_feature_type_in_bbox(bbox, 'Zoomstack_Surfacewater')
    if len(result['features']) > 0:
        return True
    else:
        return False
    
print(is_water(heathrow_bbox))
print(is_water(luton_bbox))
print(is_water(lake_bbox))
print(is_water(field_bbox))

True
True
True
False


## Buildings 

In [8]:
def is_building(bbox):
    result_local = get_feature_type_in_bbox(bbox, 'Zoomstack_LocalBuildings')
    result_district = get_feature_type_in_bbox(bbox, 'Zoomstack_DistrictBuildings')
    
    if len(result_local['features']) > 0:
        return True
    elif len(result_district['features']) > 0:
        return True
    else:
        return False

print(is_building(heathrow_bbox))
print(is_building(luton_bbox))
print(is_building(lake_bbox))
print(is_building(field_bbox))
print(is_building(forest_bbox))


True
True
False
True
False


## Railway stations

In [9]:
def is_railway_station(bbox):
    result = get_feature_type_in_bbox(bbox, 'Zoomstack_RailwayStations')
    if len(result['features']) > 0:
        return True
    else:
        return False
    

print(is_railway_station(heathrow_bbox))
print(is_railway_station(luton_bbox))
print(is_railway_station(lake_bbox))
print(is_railway_station(field_bbox))
print(is_railway_station(forest_bbox))

True
True
False
False
False


## Road

In [10]:
def is_local_road(bbox):
    result_local = get_feature_type_in_bbox(bbox, 'Zoomstack_RoadsLocal')
    result_national = get_feature_type_in_bbox(bbox, 'Zoomstack_RoadsNational')
    result_regional = get_feature_type_in_bbox(bbox, 'Zoomstack_RoadsRegional')
    
    if len(result_local['features']) > 0:
        return True
    elif len(result_national['features']) > 0:
        return True
    elif len(result_regional['features']) > 0:
        return True
    else:
        return False
    
print(is_local_road(heathrow_bbox))
print(is_local_road(luton_bbox))
print(is_local_road(lake_bbox))
print(is_local_road(field_bbox))
print(is_local_road(forest_bbox))

True
True
True
True
False


## Railway

In [11]:
def is_railway(bbox):
    result = get_feature_type_in_bbox(bbox, 'Zoomstack_Rail')
    if len(result['features']) > 0:
        return True
    else:
        return False
    
print(is_railway(heathrow_bbox))
print(is_railway(luton_bbox))
print(is_railway(lake_bbox))
print(is_railway(field_bbox))
print(is_railway(forest_bbox))

True
True
False
False
False
